<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2025notebooks/2025_0612CDP%2Bja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
filename: 2026_0612CDP+ja.ipynb
author: 浅川伸一
---

# 2025_0623 の議論

<img src="https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/refs/heads/master/2025figs/1998Zorzi_CDP_fig1.svg">
Zorzi+(1998) Fig.1 Architecture of the model. The arrow means full connectivity between layers. Each box stand for a group of letters (26) or phonemes (44).<br/>


<img src="https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/refs/heads/master/2025figs/1998Zorzi_CDP_fig8.svg">
<p>Zorzi+(1998) Fig.8. Architecture of the model with the hidden layer pathway. In both the direct pathway and the mediated pathway the layers are fully connected (arrows).</p>


orth1, orth2 --> embeddings --> phonology ($p_1,p_2,\ldots,p_n$) という流れと<br/>

```
orth1 --> emb1 ---+
                  |---> phonology
orth2 --> emb2 ---+
```


```
orth1 ---+
         |--> emb ---> phonology
orth2 ---+
```

など考えたら色々と考える必要がありそうだ。

In [6]:
import IPython
isColab = 'colab' in str(IPython.get_ipython())

import torch
device=torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
#device='cpu'
print(f'device:{device}')

# 必要なライブラリの輸入
import pandas as pd
from collections import OrderedDict

import sys
import os
import numpy as np
import time
import datetime
import operator

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

HOME = os.environ['HOME']

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

class chars_joyo():
    """
    https://ja.wikipedia.org/wiki/%E5%B8%B8%E7%94%A8%E6%BC%A2%E5%AD%97%E4%B8%80%E8%A6%A7
    常用漢字一覧（じょうようかんじいちらん）

    常用漢字は 2136 字。
    下表の配列は常用漢字表（平成22年内閣告示第2号）に準じる。
    学年の数字は、小学校学習指導要領（2017年3月告示）の学年別漢字配当表において配当されている学年を示す。
    Sは中学校以降で習うことを意味する。
    音訓は、常用漢字表に掲げられた音訓を示す。
    片仮名は音読み、平仮名は訓読みである。
    括弧でくくられた音訓は「特別なものか、又は用法のごく狭いもの」として、1字下げで示されたものである。
    ハイフンは送り仮名の付け方（昭和48年内閣告示第2号[1]）による送り仮名の区切りである。
    音訓および付表の語の学校段階（小学校・中学校・高等学校）ごとの割り振りについては、音訓の小・中・高等学校段階別割り振り表（2017年3月）を参照。
    通用字体は、常用漢字表に掲げられた「印刷文字における現代の通用字体」を示した[2]。
    手書き文字（筆写の楷書）の字形と印刷文字の字形に関しては、常用漢字表の字体・字形に関する指針 (PDF) （文化審議会国語分科会報告）を参照。
    旧字体は、『新潮日本語漢字辞典』（新潮社、2007年）の「旧字」を示した[3]。
    常用漢字表に掲げられた「いわゆる康熙字典体」とは必ずしも一致しない[4]。
    なお、表外漢字字体表の簡易慣用字体が通用字体として採用されたものについては、印刷標準字体を旧字体として示した。
    部首は康熙字典（214部）に従った。
    康熙字典にない字についても、康熙字典に倣って部首を示した。
    その際、当用漢字表を参考にした。
    画数の数え方が問題となるものは、以下の通りとした。
    「牙」……4画[5]
    「捗」……10画[6]
    「衷」……10画[7]
    「葛」……12画[6]
    「僅」……13画[6]
    「嗅」……13画[6]
    「塡」……13画[6]
    「箋」……14画[6]
    「遜」……14画[8]
    「遡」……14画[8]
    「稽」……15画[6]
    「箸」……15画[6]
    「餅」……15画[6]
    「餌」……15画[6]
    「賭」……16画[6]
    「頰」……16画[6]
    「謎」……17画[8]
    「韓」……18画[5]
    """

    def __init__(self):
        #url = 'https://raw.githubusercontent.com/cjkvi/cjkvi-tables/master/joyo2010.txt'
        #url = 'https://raw.githubusercontent.com/cjkvi/cjkvi-tables/master/jinmei2010.txt'
        url = 'https://raw.githubusercontent.com/cjkvi/cjkvi-tables/15569eaae99daef9f99f0383e9d8efbec64a7c5a/joyo2010.txt'
        joyo_fname = url.split('/')[-1]
        joyo_fname = os.path.join(os.getcwd(), 'RAM', joyo_fname)
        if os.path.exists(joyo_fname):
            joyo_df = pd.read_csv(joyo_fname, header=None, skiprows=1, delimiter='\t')
        else:
             joyo_df = pd.read_csv(url, header=None, skiprows=1, delimiter='\t')

        # カラム名の設定
        joyo_df.columns = ['通用字体', '旧字体', '総画', '学年', '追加年, 削除年', '音訓']
        #print(joyo_df.shape) # (2136, 6)

        self.char_list = joyo_df['通用字体'].to_list()
        self.df = joyo_df


device:cuda:0


In [ ]:
# 書記素の定義，書記素のうちカタカナを音韻表現としても利用

seed = 42
special_tokens = ['<PAD>', '<EOW>', '<SOW>', '<UNK>']
alphabet_upper_chars='ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'
alphabet_lower_chars='ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ'
num_chars='０１２３４５６７８９'
hira_chars='ぁあぃいぅうぇえぉおかがきぎくぐけげこごさざしじすずせぜそぞただちぢっつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもゃやゅゆょよらりるれろゎわゐゑをん'
kata_chars='ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロヮワヰヱヲンヴヵヶ'
#kata_chars=kata_chars+'一'  # カタカナ文字に伸ばし記号を加える
phon_list = list(kata_chars+'一')

# 常用漢字
joyo_chars = "".join([ch for ch in chars_joyo().char_list])

# 学習漢字 学年別
_gakushu_list = ['一右雨円王音下火花貝学気休玉金九空月犬見五口校左三山四子糸字耳七車手十出女小上森人水正生青石赤先千川早草足村大男竹中虫町天田土二日入年白八百文本名木目夕立力林六',
'引羽雲園遠黄何夏家科歌画会回海絵外角楽活間丸岩顔帰汽記弓牛魚京強教近兄形計元原言古戸午後語交光公工広考行高合国黒今才細作算姉市思止紙寺時自室社弱首秋週春書少場色食心新親図数星晴声西切雪線船前組走多太体台谷知地池茶昼朝長鳥直通弟店点電冬刀東当答頭同道読内南肉馬買売麦半番父風分聞米歩母方北妹毎万明鳴毛門夜野矢友曜用来理里話',
'悪安暗委意医育員飲院運泳駅横屋温化荷界開階寒感漢館岸期起客宮急球究級去橋業局曲銀区苦具君係軽決血研県庫湖向幸港号根祭坂皿仕使始指死詩歯事持次式実写者主取守酒受州拾終習集住重宿所暑助勝商昭消章乗植深申真神身進世整昔全想相送息速族他打対待代第題炭短談着柱注丁帳調追定庭笛鉄転登都度島投湯等豆動童農波配倍箱畑発反板悲皮美鼻筆氷表病秒品夫負部服福物平返勉放味命面問役薬油有由遊予様洋羊葉陽落流旅両緑礼列練路和',
'愛案以位囲胃衣印栄英塩央億加果課貨芽改械害街各覚完官管観関願喜器希旗機季紀議救求泣給挙漁競共協鏡極訓軍郡型径景芸欠結健建験固候功好康航告差最菜材昨刷察札殺参散産残司史士氏試児治辞失借種周祝順初唱松焼照省笑象賞信臣成清静席積折節説戦浅選然倉巣争側束続卒孫帯隊達単置仲貯兆腸低停底的典伝徒努灯働堂得特毒熱念敗梅博飯費飛必標票不付府副粉兵別変辺便包法望牧末満未脈民無約勇要養浴利陸料良量輪類令例冷歴連労老録',
'圧易移因営永衛液益演往応恩仮価可河過賀解快格確額刊幹慣眼基寄規技義逆久旧居許境興均禁句群経潔件券検険減現限個故護効厚構耕講鉱混査再妻採災際在罪財桜雑賛酸師志支枝資飼似示識質舎謝授修術述準序承招証常情条状織職制勢性政精製税績責接設絶舌銭祖素総像増造則測属損態貸退団断築張提程敵適統導銅徳独任燃能破判版犯比肥非備俵評貧婦富布武復複仏編弁保墓報豊暴貿防務夢迷綿輸余預容率略留領',
'異遺域宇映延沿我灰拡閣革割株巻干看簡危揮机貴疑吸供胸郷勤筋敬系警劇激穴憲権絹厳源呼己誤后孝皇紅鋼降刻穀骨困砂座済裁策冊蚕姿私至視詞誌磁射捨尺若樹収宗就衆従縦縮熟純処署諸除傷将障城蒸針仁垂推寸盛聖誠宣専泉洗染善創奏層操窓装臓蔵存尊宅担探誕暖段値宙忠著庁潮頂賃痛展党糖討届難乳認納脳派俳拝背肺班晩否批秘腹奮並閉陛片補暮宝訪亡忘棒枚幕密盟模訳優郵幼欲翌乱卵覧裏律臨朗論']

_l = []
for g in _gakushu_list:
    for ch in g:
        _l += ch
gakushu_chars = "".join(ch for ch in _l)

grph_list = []
for x in [hira_chars, gakushu_chars]:             # 数字は入力文字としない場合
#for x in [hira_chars, num_chars, gakushu_chars]: # 数字も入力文字とする場合
    for ch in x:
        grph_list.append(ch)
print(f'len(grph_list):{len(grph_list)}')
print(f'全書記素 grph_list:{"".join([ch for ch in grph_list])}')

print(f'len(phon_list):{len(phon_list)}')
print(f'全音素 phon_list:{phon_list}')

print(f'入力層の素子数 len(grph_list) + len(special_tokens)={len(grph_list) + len(special_tokens)}')
print(f'出力層の素子数 len(phon_list) + len(special_tokens)={len(phon_list) + len(special_tokens)}')

In [ ]:
# NTT 日本語の語彙特性単語頻度データ psylex71.txt の読み込み

if isColab:
    ntt_base = '/content'
else:
    HOME = os.environ['HOME']
    ntt_base = os.path.join(HOME, 'study/2017_2009AmanoKondo_NTTKanjiData')
psy71_fname = os.path.join(ntt_base, 'psylex71utf8_.txt')  # ファイル名
psylex71raw = open(psy71_fname, 'r').readlines()
psylex71raw = [lin.strip().split(' ')[:6] for lin in psylex71raw]   # 空白 ' ' で分離し，年度ごとの頻度を削除

# Psylex71 一行のデータは 0:共通ID, 1:独自ID, 2:表記, 3:ヨミ, 4:品詞, 5:頻度 を取り出す。
n_idx=0; n_wrd=2; n_yomi=3; n_pos=4; n_frq=5
idxes = {'n_idx':0, 'n_idx2':1, 'n_wrd':2, 'n_yomi':3, 'n_pos':4, 'n_frq':5}

kata_chars='ァアィイゥウェエォオカガキギクグケゲコゴサザシジスズセゼソゾタダチヂッツヅテデトドナニヌネノハバパヒビピフブプヘベペホボポマミムメモャヤュユョヨラリルレロヮワヰヱヲンヴヵヶ'
kata_chars=kata_chars+'一'

maxlen_grph = 2 # 書記素最大文字数 + 2 しているのは, 単語の前後に特殊トークン <SOW> <EOW> をつけるため
_psylex71_ = []
ng_yomi_words = []
dups_idx = []

valid_chars=grph_list

grph_cands=valid_chars
#phon_cands=kata_chars
phon_cands = phon_list

Psylex71 = OrderedDict()
for lin in psylex71raw:
    wrd = lin[n_wrd]
    idx = lin[n_idx]
    yomi = lin[n_yomi]
    pos = lin[n_pos]
    frq = lin[n_frq]

    if len(wrd) == maxlen_grph:  # 長さが maxlen_grph 文字である語に対して処理を行う

        # ヨミの中にカタカナ以外の文字が入っていれば NG_flag を True にする
        is_kata_yomi = True
        for p in yomi:
            if not p in kata_chars:
                is_kata_yomi = False

        # ヨミにカタカナ以外の文字が含まれていれば ng_yomi_words に加える
        if is_kata_yomi == False:
            ng_yomi_words.append((wrd,yomi))
        else:

            # valid_chars (学習漢字+)で構成されているか否かを判断
            is_valid_grph = True
            for i in range(maxlen_grph):
                if not wrd[i] in valid_chars:
                    is_valid_grph = False

            if is_valid_grph == True:
                _psylex71_.append(lin)

                if idx in Psylex71:   # すでに ID 番号が登録されていれば dups_idx リストに加える
                    dups_idx.append((idx,lin, (Psylex71[idx]['単語'],Psylex71[idx]['ヨミ'])))

                Psylex71[idx] = {'単語': wrd, 'ヨミ': yomi, '品詞': pos,'頻度': frq}


# 読み (音韻表現) の最大長値を探す
maxlen_phon = 0
for a in _psylex71_:
    if len(a[n_yomi]) > maxlen_phon:
         maxlen_phon = len(a[n_yomi])

# 結果の表示
print(f'読み込んだ psylex71.txt の単語数 len(psylex71raw):{len(psylex71raw)}')
print(f'Psylex71 の総単語数 len(_psylex71_):{len(_psylex71_)}')
print(f'作成したデータベース辞書の項目数 len(Psylex71):{len(Psylex71)}')
print(f'ヨミの最長文字数 maxlen_phon:{maxlen_phon}')
print(f'音素 (読みのカタカナ文字)数 len(phon_cands):{len(phon_cands)}')
print(f'Psylex71 におけるカタカナ以外のヨミのある単語数 len(ng_yomi_words):{len(ng_yomi_words)}')
print(f'Psylex71 における ID 番号の重複数 len(dups_idx):{len(dups_idx)}')

In [ ]:
import torch
class Psylex71_Dataset(torch.utils.data.Dataset):
    '''ニューラルネットワークモデルに Psylex71 を学習させるための PyTorch 用データセットのクラス'''

    def __init__(self,
                 dic=Psylex71,
                 grph_list=grph_list,
                 phon_list=phon_list,
                 special_tokens=special_tokens,
                 maxlen_phon=maxlen_phon +2, # ＋2 しているのは <SOW>,<EOW> という 2 つのスペシャルトークンを付加するため
                ):
        super().__init__()
        self.dic = dic
        self.special_tokens = special_tokens
        self.maxlen_phon = maxlen_phon
        self.grph_list = grph_list
        self.phon_list = phon_list
        self.input_cands = special_tokens + grph_list
        self.target_cands = special_tokens + phon_list
        self.inputs = [v['単語'] for v in dic.values()]
        self.targets = [v['ヨミ'] for v in dic.values()]

    def __len__(self):
        return len(self.dic)

    def __getitem__(self, idx):
        inp, tgt = self.inputs[idx], self.targets[idx]

        # 入力信号にも <SOW>, <EOW> トークンを付与する場合
        #inp = [self.input_cands.index('<SOW>')]  + [self.input_cands.index(x) for x in inp]  + [self.input_cands.index('<EOW>')]

        # 入力信号にはスペシャルトークンを付与しない場合
        inp = [self.input_cands.index(x) for x in inp]

        # ターゲット (教師)信号 には <SOW>, <EOW> を付与する
        tgt = [self.target_cands.index('<SOW>')] + [self.target_cands.index(x) for x in tgt] + [self.target_cands.index('<EOW>')]

        while len(tgt) < self.maxlen_phon:
            tgt = tgt + [self.target_cands.index('<PAD>')]

        inp = torch.tensor(inp, dtype=torch.int64)
        tgt = torch.tensor(tgt, dtype=torch.int64)
        return inp, tgt

    def getitem(self, idx):
        wrd = self.inputs[idx]
        phn = self.targets[idx]
        return wrd, phn

    def ids2argmax(self, ids):
        out = np.array([torch.argmax(idx).numpy() for idx in ids], dtype=np.int32)
        return out

    def ids2tgt(self, ids):
        #out = [self.target_cands[torch.argmax(idx)] for idx in ids]
        out = [self.target_cands[idx] for idx in ids]
        return out

    def ids2inp(self, ids):
        out = [self.input_cands[idx] for idx in ids]
        return out

    def target_ids2target(self, ids:list):
        ret = []
        for idx in ids:
            if idx == self.target_cands.index('<EOW>'):
                return ret+['<EOW>']
            ret.append(self.target_cands[idx])
        return ret


psylex71_ds = Psylex71_Dataset()
for N in np.random.permutation(psylex71_ds.__len__())[:15]:
    inp, tgt = psylex71_ds.__getitem__(N)
    print(f'psylex71_ds.ids2inp(inp):{psylex71_ds.ids2inp(inp)}',
          f'{inp.numpy()}',
          f'psylex71_ds.ids2tgt(tgt):{psylex71_ds.ids2tgt(tgt)}',
          f'{tgt.numpy()}')


train_size = int(psylex71_ds.__len__() * 0.7)
valid_size = psylex71_ds.__len__() - train_size
train_ds, valid_ds = torch.utils.data.random_split(dataset=psylex71_ds, lengths=(train_size, valid_size), generator=torch.Generator().manual_seed(seed))
#print(train_ds.__len__(), valid_ds.__len__())

batch_size = 512
train_dl = torch.utils.data.DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(dataset=valid_ds, batch_size=batch_size, shuffle=False)

def _collate_fn(batch):
    inps, tgts = list(zip(*batch))
    inps = list(inps)
    tgts = list(tgts)
    return inps, tgts

# batch_size = 4
train_dl = torch.utils.data.DataLoader(
    dataset=train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=_collate_fn)

valid_dl = torch.utils.data.DataLoader(
    dataset=valid_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=_collate_fn)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

class TLA(torch.nn.Module):
    def __init__(self,
                 # maxlen_phon+2 しているのは単語の前後に <SOW>, <EOW> トークンを付けるため
                 inp_size= (len(grph_list)+len(special_tokens)), # * (maxlen_grph + 2),
                 inp_len=maxlen_grph, #  + 2,
                 out_size=len(phon_list)+len(special_tokens),
                 out_len=maxlen_phon+2,
                 hid_size=128,
                 device='cpu',
                ):
        super().__init__()
        self.inp_size=inp_size
        self.inp_len=inp_len
        self.out_size=out_size
        self.out_len=out_len
        self.hid_size=hid_size

        self.emb_layers = [torch.nn.Embedding(num_embeddings=inp_size, embedding_dim=hid_size, padding_idx=0).to(device) for _ in range(inp_len)]
        #self.emb_layer = torch.nn.Embedding(num_embeddings=inp_size, embedding_dim=hid_size, padding_idx=0).to(device)

        self.hid_layer = torch.nn.Linear(in_features=hid_size * inp_len, out_features=hid_size).to(device)
        #self.hid_layer = torch.nn.Linear(in_features=inp_len * inp_size, out_features=hid_size)

        self.out_layers = [torch.nn.Linear(in_features=hid_size, out_features=out_size).to(device) for _ in range(out_len)]

    def forward(self, inp):
        X = inp
        batch_size = X.size(0)
        n_grph = X.size(1)

        embs = []
        for i in range(n_grph):
            _emb = self.emb_layers[i](X[:,i])
            #print(f'{i}:_emb.size():{_emb.size()}')
            embs.append(_emb)

        _embs = torch.concat(embs,dim=1)
        X = _embs
        X = self.hid_layer(X)         # 中間層次元へ変換

        # 出力層の音韻表現ごとへ変換
        outputs = []
        for i in range(self.out_len):
            _out = self.out_layers[i](X)
            outputs.append(_out)

        # softmax 変換
        outputs = [torch.nn.functional.softmax(out,dim=1) for out in outputs]

        O = torch.empty(self.out_len, batch_size, self.out_size)
        for i in range(len(outputs)):
            O[i] = outputs[i]
        O = O.reshape(batch_size, self.out_len, self.out_size)
        return O

tla = TLA(device=device)
tla.eval()

# idx に整数を指定して,対応するデータを取得する
idx = np.random.choice(train_ds.__len__())

# データセットから返ってくる値は入力信号 inp と教師信号 tch
inp, tch = train_ds.__getitem__(idx)
print(f'idx:{idx}:', f'inp:{inp}', f'tch:{tch}')

# 入出力信号はトークン ID 番号であるため人間が読みやすいように変換して表示
print(f'train_ds.dataset.ids2inp({inp}):{train_ds.dataset.ids2inp(inp)}')
print(f'train_ds.dataset.ids2tgt({tch}):{train_ds.dataset.ids2tgt(tch)}')

inp = pad_sequence(inp.unsqueeze(0), batch_first=True).to(device)

outs = tla(inp)
outs = [out.cpu() for out in outs]
print('出力:', train_ds.dataset.ids2tgt([int(_out.argmax().numpy()) for _out in outs[0]]), end=": ")
print('出力 ids:', [int(_out.argmax().numpy()) for _out in outs[0]])
#print('出力 ids:', [int(out.argmax().numpy()) for out in outs])

tch = tch.cpu()
print('教師:', train_ds.dataset.ids2tgt([idx.numpy() for idx in tch]), end=": ")
print('教師 ids:', [int(_tch.numpy()) for _tch in tch])

In [ ]:
# ミニバッチバージョン
loss_f = torch.nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(tla.parameters(), lr=1e-6)
tla.train()
epochs = 3
for epoch in range(epochs):
    sum_loss = 0.
    for inps, tchs in tqdm(train_dl):
        inps = pad_sequence(inps, batch_first=True).to(device)
        tchs = pad_sequence(tchs, batch_first=True)
        outs = tla(inps)

        sum_loss = 0.
        loss = 0.
        for j in range(len(tchs)):
            loss += loss_f(outs[j],tchs[j])
        sum_loss += loss.item()
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        out_ids = [out.argmax(dim=1) for out in outs]
        count  = 0
        for tch, out in zip(tchs[:], out_ids[:]):
            yesno = ((tch==out) * 1).sum().numpy() == len(tch)
            count += 1 if yesno else 0
        p_correct = count / len(tch)
        #print('出力 ids:', [int(out.argmax().numpy()) for out in outs[:10]])
        #print('教師 ids:', [tch  for tch in tchs[:3]])

    print(f'p_correct:{p_correct:.3f}', end=": ")
    #epoch_loss += sum_loss.item()
    print(f'sum_loss:{sum_loss:.3f}')


In [ ]:
tla.eval()

# idx に整数を指定して,対応するデータを取得する
idx = np.random.choice(train_ds.__len__())

# データセットから返ってくる値は入力信号 inp と教師信号 tch
inp, tch = train_ds.__getitem__(idx)
print(f'idx:{idx}:', f'inp:{inp}', f'tch:{tch}')

# 入出力信号はトークン ID 番号であるため人間が読みやすいように変換して表示
print(f'train_ds.dataset.ids2inp({inp}):{train_ds.dataset.ids2inp(inp)}')
print(f'train_ds.dataset.ids2tgt({tch}):{train_ds.dataset.ids2tgt(tch)}')

inp = pad_sequence(inp.unsqueeze(0), batch_first=True).to(device)

outs = tla(inp)
outs = [out.cpu() for out in outs]
print('出力:', train_ds.dataset.ids2tgt([int(_out.argmax().numpy()) for _out in outs[0]]), end=": ")
print('出力 ids:', [int(_out.argmax().numpy()) for _out in outs[0]])
#print('出力 ids:', [int(out.argmax().numpy()) for out in outs])

tch = tch.cpu()
print('教師:', train_ds.dataset.ids2tgt([idx.numpy() for idx in tch]), end=": ")
print('教師 ids:', [int(_tch.numpy()) for _tch in tch])